In [ ]:
# default_exp preprocessing.clean

In [ ]:
#all_slow

# Clean

> Functions to split the raw EHR dataset, clean and save for further processing & vocab creation.

In [ ]:
#hide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
from lemonpie.basics import *
from fastai.imports import *

In [ ]:
#hide
from nbdev.showdoc import *

## Split 

Once dataset is assembled, the folder will look as follows .. 

In [ ]:
DATA_STORE

'/home/vinod/.lemonpie/datasets'

In [ ]:
PATH_1K

'/home/vinod/.lemonpie/datasets/synthea/1K'

In [ ]:
os.listdir(f'{PATH_1K}/raw_original')

['patients.csv',
 'observations.csv',
 'allergies.csv',
 'payers.csv',
 'careplans.csv',
 'medications.csv',
 'devices.csv',
 'organizations.csv',
 'imaging_studies.csv',
 '.~lock.conditions.csv#',
 'procedures.csv',
 'payer_transitions.csv',
 'supplies.csv',
 'conditions.csv',
 'providers.csv',
 'encounters.csv',
 'immunizations.csv']

In [ ]:
#export
def read_raw_ehrdata(path, csv_names = FILENAMES):
    '''Read raw EHR data'''
    dfs = [pd.read_csv(f'{path}/{fname}.csv', low_memory=False) for fname in csv_names]
    return dfs    

In [ ]:
dfs = read_raw_ehrdata(f'{PATH_1K}/raw_original')

In [ ]:
patients, observations, allergies, careplans, medications, imaging_studies, procedures, conditions, immunizations = dfs

In [ ]:
#export
def split_patients(patients, valid_pct=0.2, test_pct=0.2, random_state=1234):
    '''Split the patients dataframe'''
    train_pct = 1 - (valid_pct + test_pct)
    print(f'Splits:: train: {train_pct}, valid: {valid_pct}, test: {test_pct}')
    patients = patients.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return np.split(patients, [int(train_pct*len(patients)), int((train_pct+valid_pct)*len(patients))])

In [ ]:
train, valid, test = split_patients(patients, .2,.1)

Splits:: train: 0.7, valid: 0.2, test: 0.1


In [ ]:
len(patients), len(train), len(valid), len(test)

(1171, 819, 234, 118)

In [ ]:
assert len(patients) == len(train)+len(valid)+len(test)

In [ ]:
#export
def split_ehr_dataset(path, valid_pct=0.2, test_pct=0.2, random_state=1234):
    '''Split EHR dataset into train, valid, test and save'''

    train_dfs, valid_dfs, test_dfs = [],[],[]
    
    dfs = read_raw_ehrdata(f'{path}/raw_original')
    all_pts = dfs[0]
    all_pts.rename(str.lower, axis='columns', inplace=True)
    train_pt, valid_pt, test_pt = split_patients(dfs[0], valid_pct, test_pct, random_state)
    train_dfs.append(train_pt)
    valid_dfs.append(valid_pt)
    test_dfs.append(test_pt)
    print(f'Split {FILENAMES[0]} into:: Train: {len(train_pt)}, Valid: {len(valid_pt)}, Test: {len(test_pt)} -- Total before split: {len(dfs[0])}')
    
    for df, name in zip(dfs[1:], FILENAMES[1:]):
        df = df.set_index('PATIENT')
        df_train = df.loc[df.index.intersection(train_pt['id']).unique()]
        df_valid = df.loc[df.index.intersection(valid_pt['id']).unique()]
        df_test = df.loc[df.index.intersection(test_pt['id']).unique()]
        assert len(df) == len(df_train)+len(df_valid)+len(df_test),f'Split failed {name}: {len(df)} != {len(df_train)}+{len(df_valid)}+{len(df_test)}'
        train_dfs.append(df_train.reset_index())
        valid_dfs.append(df_valid.reset_index())
        test_dfs.append(df_test.reset_index())

    
    for split in ['train', 'valid', 'test']:
        d = Path(f'{path}/raw_split/{split}')
        d.mkdir(parents=True, exist_ok=True)
        
        if split == 'train':
            for df, name in zip(train_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved train data to {d}')
        
        if split == 'valid':
            for df, name in zip(valid_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved valid data to {d}')
    
        if split == 'test':
            for df, name in zip(test_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved test data to {d}')

In [ ]:
split_ehr_dataset(PATH_1K) #will use default values for split percents

Splits:: train: 0.6, valid: 0.2, test: 0.2
Split patients into:: Train: 702, Valid: 234, Test: 235 -- Total before split: 1171
Saved train data to /home/vinod/.lemonpie/datasets/synthea/1K/raw_split/train
Saved valid data to /home/vinod/.lemonpie/datasets/synthea/1K/raw_split/valid
Saved test data to /home/vinod/.lemonpie/datasets/synthea/1K/raw_split/test


## Clean

In [ ]:
#export
def cleanup_pts(pts, is_train, today=None):
    '''Clean patients df'''
    
    pts.rename(str.lower, axis='columns', inplace=True)
    pts = pts.loc[:, ['id', 'birthdate', 'marital', 'race', 'ethnicity', 'gender', 'birthplace', 'city', 'state', 'zip']]
    pts.rename(columns={"id":"patient"}, inplace=True)
    pts = pts.astype({'birthdate':'datetime64'}) 
    pts['zip'] = pts['zip'].fillna(0.0).astype(int)    
    if today == None: today = pd.Timestamp.today()
    else            : today = pd.to_datetime(today)
    pts['age_now_days'] = pts['birthdate'].apply(lambda bday: (today-bday).days)
    
    pts.fillna('xxxnan', inplace=True)
    if is_train: pt_codes = pts.drop(columns=['patient'], inplace=False)
    pts.set_index('patient', inplace=True)
    pt_demographics = pts
    patients = pts.loc[:,['birthdate']]
    
    return [patients, pt_demographics, pt_codes] if is_train else [patients, pt_demographics]

`patients` data frame looks like this before cleanup.. 

In [ ]:
patients.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,NaN,999-76-6866,S99984236,X19277260X,Mr.,José Eduardo181,Gómez206,NaN,...,Marigot Saint Andrew Parish DM,427 Balistreri Way Unit 19,Chicopee,Massachusetts,Hampden County,1013.0,42.228354,-72.562951,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,NaN,999-73-5361,S99962402,X88275464X,Mr.,Milo271,Feil794,NaN,...,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,NaN,999-27-3385,S99972682,X73754411X,Mr.,Jayson808,Fadel536,NaN,...,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,Hampden County,1020.0,42.181642,-72.608842,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,NaN,999-85-4926,S99974448,X40915583X,Mrs.,Mariana775,Rutherford999,NaN,...,Yarmouth Massachusetts US,999 Kuhn Forge,Lowell,Massachusetts,Middlesex County,1851.0,42.636143,-71.343255,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,NaN,999-60-7372,S99915787,X86772962X,Mr.,Gregorio366,Auer97,NaN,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,598763.07,3772.20


In [ ]:
pt_data = cleanup_pts(patients, is_train=True, today=SYNTHEA_DATAGEN_DATES['1K'])
patients, pt_demo, pt_codes = pt_data[0], pt_data[1], pt_data[2]

Our cleanup function produces the following 3 dfs - `patients`, `pt_demographics`, `pt_codes`

In [ ]:
for df in pt_data:
    display(df.head())

,birthdate
patient,
1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25
034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14
10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27
f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18


,birthdate,marital,race,ethnicity,gender,birthplace,city,state,zip,age_now_days
patient,,,,,,,,,,
1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,M,white,hispanic,M,Marigot Saint Andrew Parish DM,Chicopee,Massachusetts,1013,11617
034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,M,white,nonhispanic,M,Danvers Massachusetts US,Somerville,Massachusetts,2143,13636
10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,M,white,nonhispanic,M,Springfield Massachusetts US,Chicopee,Massachusetts,1020,10513
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,M,white,nonhispanic,F,Yarmouth Massachusetts US,Lowell,Massachusetts,1851,15633
f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,xxxnan,white,nonhispanic,M,Patras Achaea GR,Boston,Massachusetts,2135,8914


,birthdate,marital,race,ethnicity,gender,birthplace,city,state,zip,age_now_days
0,1989-05-25,M,white,hispanic,M,Marigot Saint Andrew Parish DM,Chicopee,Massachusetts,1013,11617
1,1983-11-14,M,white,nonhispanic,M,Danvers Massachusetts US,Somerville,Massachusetts,2143,13636
2,1992-06-02,M,white,nonhispanic,M,Springfield Massachusetts US,Chicopee,Massachusetts,1020,10513
3,1978-05-27,M,white,nonhispanic,F,Yarmouth Massachusetts US,Lowell,Massachusetts,1851,15633
4,1996-10-18,xxxnan,white,nonhispanic,M,Patras Achaea GR,Boston,Massachusetts,2135,8914


**The case for keeping a record of the data generation date**

Also note the difference in `age_now` if it were calculated based on default (`pd.Timestamp.today()`) vs `SYNTHEA_DATAGEN_DATES['1K']` which is the data generation date for this 1K dataset.  

In [ ]:
# age_now -- today vs run_date
(pd.to_datetime(pd.Timestamp.today()) - patients.iloc[2])[0].days, (pd.to_datetime(SYNTHEA_DATAGEN_DATES['1K']) - patients.iloc[2])[0].days

(10522, 10513)

In [ ]:
pt_codes.marital.unique()

array(['M', 'xxxnan', 'S'], dtype=object)

In [ ]:
#export
def cleanup_obs(obs, is_train):
    '''Clean observations df'''
    
    obs.rename(str.lower, axis='columns', inplace=True)
    obs.units.fillna('xxxnan', inplace=True)
    obs.dropna(subset=['value'], inplace=True)
    
    obs.rename(columns={"code":"orig_code", "description":"desc"}, inplace=True)
    obs['code'] = obs['orig_code'].str.cat(obs[['value', 'units', 'type']].astype(str), sep='||')

    if is_train: obs_codes = obs.loc[:, ['orig_code', 'desc', 'value', 'units', 'type']]
    
    obs = obs.loc[:, ['patient', 'date', 'code']]
    obs = obs.astype({'date':'datetime64'})
    obs.set_index('patient', inplace=True)
    
    return [obs, obs_codes] if is_train else [obs]

- Drops rows with null in the `VALUE` column
- Creates a new `code` column with a concatenation of `code`, `value`, `units` and `type`
    - so that we can use the following logic during vocab creation for observations (further detailed in the vocab documentation)

**For `numeric`**
```
for 'numeric'
    get unique 'codes'
    for each unique code
        get unique 'units'
            for each unique unit
                bucketize 'values'
                create vocab entry for each 'bucket' -- code||value_bucket||units
```
**For `text`**
```
for 'text'
    get unique 'codes'
    for each unique code
        get unique 'units' #this will be null
            for each unique unit
                get unique 'values'
                create vocab entry for each -- code||value||units
```

'observations' df before cleanup ..

In [ ]:
observations.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8302-2,Body Height,193.3,cm,numeric
1,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score},numeric
2,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,29463-7,Body Weight,87.8,kg,numeric
3,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,39156-5,Body Mass Index,23.5,kg/m2,numeric
4,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8462-4,Diastolic Blood Pressure,82.0,mm[Hg],numeric


In [ ]:
obs_data = cleanup_obs(observations, is_train=True)

after cleanup..

In [ ]:
for df in obs_data:
    display(df.head())

,date,code
patient,,
034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23 17:45:28,8302-2||193.3||cm||numeric
034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23 17:45:28,72514-3||2.0||{score}||numeric
034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23 17:45:28,29463-7||87.8||kg||numeric
034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23 17:45:28,39156-5||23.5||kg/m2||numeric
034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23 17:45:28,8462-4||82.0||mm[Hg]||numeric


,orig_code,desc,value,units,type
0,8302-2,Body Height,193.3,cm,numeric
1,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score},numeric
2,29463-7,Body Weight,87.8,kg,numeric
3,39156-5,Body Mass Index,23.5,kg/m2,numeric
4,8462-4,Diastolic Blood Pressure,82.0,mm[Hg],numeric


In [ ]:
#export
def cleanup_algs(allergies, is_train):
    '''Clean allergies df'''
    
    allergies.rename(str.lower, axis='columns', inplace=True)
    allergies.drop(columns=['encounter'], inplace=True)
    
    stops = pd.DataFrame(allergies.loc[allergies['stop'].notnull(),:])
    allergies['code'] = allergies['code'].apply(lambda x: f'{str(x)}||START')
    stops['code'] = stops['code'].apply(lambda x: f'{str(x)}||STOP')
    allergies.drop(columns=['stop'], inplace=True)
    stops.drop(columns=['start'], inplace=True)
    allergies.rename(columns={"start":"date", "description":"desc"}, inplace=True)
    stops.rename(columns={"stop":"date", "description":"desc"}, inplace=True)
    allergies = allergies.append(stops, ignore_index=True)
    
    if is_train: alg_codes = allergies.loc[:, ['code', 'desc']]
        
    allergies.drop(columns=['desc'], inplace=True)
    allergies = allergies.astype({'date':'datetime64'})
    allergies.set_index('patient', inplace=True)
    return [allergies, alg_codes] if is_train else [allergies]

`allergies` have a start and stop date in the same row indicating when an allergy (indicated by its code) started and stopped (or not) for a patient. <br>
So in the cleanup, we flatten that out, meaning create new rows for stop dates. <br>
The dataframe looks as follows before cleanup..

In [ ]:
allergies.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1982-10-25,NaN,76982e06-f8b8-4509-9ca3-65a99c8650fe,b896bf40-8b72-42b7-b205-142ee3a56b55,300916003,Latex allergy
1,1982-10-25,NaN,76982e06-f8b8-4509-9ca3-65a99c8650fe,b896bf40-8b72-42b7-b205-142ee3a56b55,300913006,Shellfish allergy
2,2002-01-25,NaN,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,7be1a590-4239-4826-9872-031327f3c368,419474003,Allergy to mould
3,2002-01-25,NaN,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,7be1a590-4239-4826-9872-031327f3c368,232347008,Dander (animal) allergy
4,2002-01-25,NaN,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,7be1a590-4239-4826-9872-031327f3c368,418689008,Allergy to grass pollen


In [ ]:
alg_data = cleanup_algs(allergies, is_train=True)

Resulting in the following output after cleanup.. 

In [ ]:
for df in alg_data:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
76982e06-f8b8-4509-9ca3-65a99c8650fe,1982-10-25,300916003||START
76982e06-f8b8-4509-9ca3-65a99c8650fe,1982-10-25,300913006||START
71ba0469-f0cc-4177-ac70-ea07cb01c8b8,2002-01-25,419474003||START


,date,code
patient,,
96942a16-75bc-4026-bd63-e985b0ca1d6d,2016-09-18,418689008||STOP
96942a16-75bc-4026-bd63-e985b0ca1d6d,2016-09-18,419263009||STOP
e6ff4bf9-09c2-4976-aa84-cca142207cf8,2016-06-25,300916003||STOP


,code,desc
0,300916003||START,Latex allergy
1,300913006||START,Shellfish allergy
2,419474003||START,Allergy to mould


,code,desc
658,418689008||STOP,Allergy to grass pollen
659,419263009||STOP,Allergy to tree pollen
660,300916003||STOP,Latex allergy


In [ ]:
#export
def cleanup_crpls(careplans, is_train):
    '''Clean careplans df'''
    
    careplans.rename(str.lower, axis='columns', inplace=True)
    careplans = careplans.loc[:, ['start', 'stop', 'patient', 'code', 'description']]
    
    stops = pd.DataFrame(careplans.loc[careplans['stop'].notnull(),:])
    careplans['code'] = careplans['code'].apply(lambda x: f'{str(x)}||START')
    stops['code'] = stops['code'].apply(lambda x: f'{str(x)}||STOP')
    careplans.drop(columns=['stop'], inplace=True)
    stops.drop(columns=['start'], inplace=True)
    careplans.rename(columns={"start":"date", "description":"desc"}, inplace=True)
    stops.rename(columns={"stop":"date", "description":"desc"}, inplace=True)
    careplans = careplans.append(stops, ignore_index=True)
    
    if is_train: crpl_codes = careplans.loc[:, ['code', 'desc']]

    careplans.drop(columns=['desc'], inplace=True)
    careplans = careplans.astype({'date':'datetime64'})
    careplans.set_index('patient', inplace=True)
    return [careplans, crpl_codes] if is_train else [careplans]

In [ ]:
careplans.head()

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,225358003,Wound care,284551006.0,Laceration of foot


In [ ]:
crpl_data = cleanup_crpls(careplans, is_train=True)

In [ ]:
for df in crpl_data:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
034e9e3b-2def-4559-bb2a-7850888ae060,2010-01-23,53950000||START
10339b10-3cd1-4ac3-ac13-ec26728cb592,2011-05-13,53950000||START
f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,2011-12-31,53950000||START


,date,code
patient,,
6d048a56-edb8-4f29-891d-7a84d75a8e78,2002-11-30,53950000||STOP
fca3178e-fb68-41c3-8598-702d3ca68b96,1983-09-29,91251008||STOP
fca3178e-fb68-41c3-8598-702d3ca68b96,1984-11-22,385691007||STOP


,code,desc
0,53950000||START,Respiratory therapy
1,53950000||START,Respiratory therapy
2,53950000||START,Respiratory therapy


,code,desc
5431,53950000||STOP,Respiratory therapy
5432,91251008||STOP,Physical therapy procedure
5433,385691007||STOP,Fracture care


In [ ]:
#export
def cleanup_meds(medications, is_train):
    '''Clean `medications` df'''
    
    medications.rename(str.lower, axis='columns', inplace=True)
    medications = medications.loc[:, ['start', 'stop', 'patient', 'code', 'description']]
    
    stops = pd.DataFrame(medications.loc[medications['stop'].notnull(),:])
    medications['code'] = medications['code'].apply(lambda x: f'{str(x)}||START')
    stops['code'] = stops['code'].apply(lambda x: f'{str(x)}||STOP')
    medications.drop(columns=['stop'], inplace=True)
    stops.drop(columns=['start'], inplace=True)
    medications.rename(columns={"start":"date", "description":"desc"}, inplace=True)
    stops.rename(columns={"stop":"date", "description":"desc"}, inplace=True)
    medications = medications.append(stops, ignore_index=True)
    
    if is_train: med_codes = medications.loc[:, ['code', 'desc']]

    medications.drop(columns=['desc'], inplace=True)
    medications = medications.astype({'date':'datetime64'})
    medications.set_index('patient', inplace=True)
    return [medications, med_codes] if is_train else [medications]

In [ ]:
medications.head()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2010-05-05T00:26:23Z,2011-04-30T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,Etonogestrel 68 MG Drug Implant,677.08,0.0,12,8124.96,NaN,NaN
1,2011-04-30T00:26:23Z,2012-04-24T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,Etonogestrel 68 MG Drug Implant,624.09,0.0,12,7489.08,NaN,NaN
2,2012-04-24T00:26:23Z,2013-04-19T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,Yaz 28 Day Pack,43.32,0.0,12,519.84,NaN,NaN
3,2011-05-13T12:58:08Z,2011-05-27T12:58:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,Acetaminophen 325 MG Oral Tablet,8.14,0.0,1,8.14,10509002.0,Acute bronchitis (disorder)
4,2011-12-08T15:02:18Z,2011-12-22T15:02:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,Amoxicillin 250 MG / Clavulanate 125 MG Oral T...,11.91,0.0,1,11.91,444814009.0,Viral sinusitis (disorder)


In [ ]:
med_data = cleanup_meds(medications, is_train=True)

In [ ]:
for df in med_data:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,2010-05-05 00:26:23,389221||START
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,2011-04-30 00:26:23,389221||START
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,2012-04-24 00:26:23,748856||START


,date,code
patient,,
6d048a56-edb8-4f29-891d-7a84d75a8e78,2005-12-31 17:27:52,2123111||STOP
fca3178e-fb68-41c3-8598-702d3ca68b96,1983-09-29 17:27:52,243670||STOP
fca3178e-fb68-41c3-8598-702d3ca68b96,1984-11-22 17:27:52,313782||STOP


,code,desc
0,389221||START,Etonogestrel 68 MG Drug Implant
1,389221||START,Etonogestrel 68 MG Drug Implant
2,748856||START,Yaz 28 Day Pack


,code,desc
84080,2123111||STOP,NDA020503 200 ACTUAT Albuterol 0.09 MG/ACTUAT ...
84081,243670||STOP,Aspirin 81 MG Oral Tablet
84082,313782||STOP,Acetaminophen 325 MG Oral Tablet


In [ ]:
#export
def cleanup_img(imaging_studies, is_train):
    '''Clean `imaging` df'''
    
    imaging_studies.rename(str.lower, axis='columns', inplace=True)
    imaging_studies.rename(columns={"bodysite_code":"code", "bodysite_description":"desc"}, inplace=True)
    if is_train: img_codes = imaging_studies.loc[:, ['code', 'desc']]
        
    imaging_studies = imaging_studies.loc[:, ['patient', 'date', 'code']]
    imaging_studies = imaging_studies.astype({'date':'datetime64'})
    imaging_studies.set_index('patient', inplace=True)
    return [imaging_studies, img_codes] if is_train else [imaging_studies]

In [ ]:
imaging_studies.head()

,Id,DATE,PATIENT,ENCOUNTER,BODYSITE_CODE,BODYSITE_DESCRIPTION,MODALITY_CODE,MODALITY_DESCRIPTION,SOP_CODE,SOP_DESCRIPTION
0,d3e49b38-7634-4416-879d-7bc68bf3e7df,2014-07-08T15:35:36Z,b58731cc-2d8b-4c2d-b327-4cab771af3ef,3a36836d-da25-4e73-808b-972b669b7e4e,40983000,Arm,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage
1,46baf530-4941-40ab-8219-685a08fd9086,2014-01-22T18:58:37Z,2ffe9369-24e4-414b-8973-258fad09313a,33b71e4b-0690-4fe9-897a-dc3b2ff9215c,40983000,Arm,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage
2,b8fb8a6e-a2f5-46c9-8b3f-a35aa982efcd,2001-12-01T02:08:27Z,86b97fc7-ae8f-4e0d-8e66-db68f36e7a76,e42d1046-568d-46c2-b0a5-d910b2f3bd1d,8205005,Wrist,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage
3,10c8a016-4504-4653-bddf-2dd3610886c8,2004-07-03T20:46:46Z,71ba0469-f0cc-4177-ac70-ea07cb01c8b8,323fca87-817f-4d58-8486-ba92ea739399,51299004,Clavicle,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage
4,4221534c-d379-4c6b-a22e-d7eae3fa2609,2017-02-08T08:42:44Z,d49f748f-928d-40e8-92c8-73e4c5679711,cfef48b3-b769-4794-a3e7-f57f7ba8d387,344001,Ankle,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage


In [ ]:
img_data = cleanup_img(imaging_studies, is_train=True)

In [ ]:
for df in img_data:
    display(df.head(3))

,date,code
patient,,
b58731cc-2d8b-4c2d-b327-4cab771af3ef,2014-07-08 15:35:36,40983000
2ffe9369-24e4-414b-8973-258fad09313a,2014-01-22 18:58:37,40983000
86b97fc7-ae8f-4e0d-8e66-db68f36e7a76,2001-12-01 02:08:27,8205005


,code,desc
0,40983000,Arm
1,40983000,Arm
2,8205005,Wrist


In [ ]:
#export
def cleanup_procs(procedures, is_train):
    '''Clean `procedures` df'''
    
    procedures.rename(str.lower, axis='columns', inplace=True)
    procedures.rename(columns={"description":"desc"}, inplace=True)
    if is_train: proc_codes = procedures.loc[:, ['code', 'desc']]
    
    procedures = procedures.loc[:, ['patient', 'date', 'code']]
    procedures = procedures.astype({'date':'datetime64'})
    procedures.set_index('patient', inplace=True)
    return [procedures, proc_codes] if is_train else [procedures]

In [ ]:
procedures.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2011-04-30T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,169553002,Insertion of subcutaneous contraceptive,14896.56,NaN,NaN
1,2010-07-27T12:58:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,430193006,Medication Reconciliation (procedure),726.51,NaN,NaN
2,2010-11-20T03:04:34Z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,430193006,Medication Reconciliation (procedure),788.50,NaN,NaN
3,2011-02-07T03:04:34Z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,117015009,Throat culture (procedure),2070.44,195662009.0,Acute viral pharyngitis (disorder)
4,2011-04-19T03:04:34Z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,640837d9-845a-433c-9fad-47426664a69d,117015009,Throat culture (procedure),2479.39,195662009.0,Acute viral pharyngitis (disorder)


In [ ]:
proc_data = cleanup_procs(procedures, is_train=True)

In [ ]:
for df in proc_data:
    display(df.head(3))

,date,code
patient,,
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,2011-04-30 00:26:23,169553002
10339b10-3cd1-4ac3-ac13-ec26728cb592,2010-07-27 12:58:08,430193006
f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,2010-11-20 03:04:34,430193006


,code,desc
0,169553002,Insertion of subcutaneous contraceptive
1,430193006,Medication Reconciliation (procedure)
2,430193006,Medication Reconciliation (procedure)


In [ ]:
#export
def cleanup_cnds(conditions, is_train):
    '''Clean `conditions` df'''
    
    conditions.rename(str.lower, axis='columns', inplace=True)
    conditions.drop(columns=['encounter'], inplace=True)
    stops = pd.DataFrame(conditions.loc[conditions['stop'].notnull(),:])
    conditions['code'] = conditions['code'].apply(lambda x: f'{str(x)}||START')
    stops['code'] = stops['code'].apply(lambda x: f'{str(x)}||STOP')
    conditions.drop(columns=['stop'], inplace=True)
    stops.drop(columns=['start'], inplace=True)
    conditions.rename(columns={"start":"date", "description":"desc"}, inplace=True)
    stops.rename(columns={"stop":"date","description":"desc"}, inplace=True)
    conditions = conditions.append(stops, ignore_index=True)
        
    if is_train: cnd_codes = conditions.loc[:, ['code', 'desc']]
        
    conditions.drop(columns=['desc'], inplace=True)
    conditions = conditions.astype({'date':'datetime64'})
    conditions.set_index('patient', inplace=True)
    return [conditions, cnd_codes] if is_train else [conditions]

In [ ]:
conditions.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2001-05-01,NaN,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,Chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,444814009,Viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,195662009,Acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,10509002,Acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,195662009,Acute viral pharyngitis (disorder)


In [ ]:
cnd_data = cleanup_cnds(conditions, is_train=True)

In [ ]:
for df in cnd_data:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
1d604da9-9a81-4ba9-80c2-de3375d59b40,2001-05-01,40055000||START
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,2011-08-09,444814009||START
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,2011-11-16,195662009||START


,date,code
patient,,
fca3178e-fb68-41c3-8598-702d3ca68b96,1986-03-02,43878008||STOP
fc817953-cc8b-45db-9c85-7c0ced8fa90d,2010-11-25,444814009||STOP
fc817953-cc8b-45db-9c85-7c0ced8fa90d,2012-05-14,444814009||STOP


,code,desc
0,40055000||START,Chronic sinusitis (disorder)
1,444814009||START,Viral sinusitis (disorder)
2,195662009||START,Acute viral pharyngitis (disorder)


,code,desc
12938,43878008||STOP,Streptococcal sore throat (disorder)
12939,444814009||STOP,Viral sinusitis (disorder)
12940,444814009||STOP,Viral sinusitis (disorder)


In [ ]:
#export
def cleanup_immns(immunizations, is_train):
    '''Clean `immunizations` df'''
    
    immunizations.rename(str.lower, axis='columns', inplace=True)
    immunizations.rename(columns={"description":"desc"}, inplace=True)
    if is_train: imm_codes = immunizations.loc[:, ['code', 'desc']]
        
    immunizations = immunizations.loc[:, ['patient', 'date', 'code']]
    immunizations = immunizations.astype({'date':'datetime64'})
    immunizations.set_index('patient', inplace=True)
    return [immunizations, imm_codes] if is_train else [immunizations]

In [ ]:
immunizations.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST
0,2010-07-27T12:58:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,140,Influenza seasonal injectable preservative ...,140.52
1,2010-11-20T03:04:34Z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,140,Influenza seasonal injectable preservative ...,140.52
2,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,140,Influenza seasonal injectable preservative ...,140.52
3,2011-11-26T03:04:34Z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1923c698-accd-4d70-ba09-e1938f6e96d1,140,Influenza seasonal injectable preservative ...,140.52
4,2011-07-28T15:02:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,b85c339a-6076-43ed-b9d0-9cf013dec49d,140,Influenza seasonal injectable preservative ...,140.52


In [ ]:
imm_data = cleanup_immns(immunizations, is_train=True)

In [ ]:
for df in imm_data:
    display(df.head(3))

,date,code
patient,,
10339b10-3cd1-4ac3-ac13-ec26728cb592,2010-07-27 12:58:08,140
f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,2010-11-20 03:04:34,140
034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23 17:45:28,140


,code,desc
0,140,Influenza seasonal injectable preservative ...
1,140,Influenza seasonal injectable preservative ...
2,140,Influenza seasonal injectable preservative ...


### Clean all

In [ ]:
#export
def cleanup_dataset(path, is_train, today=None):
    '''Clean all dfs in a split'''
    dfs = read_raw_ehrdata(path)
 
    pt_data   = cleanup_pts(dfs[0],   is_train, today)
    obs_data  = cleanup_obs(dfs[1],   is_train)
    alg_data  = cleanup_algs(dfs[2],  is_train)
    crpl_data = cleanup_crpls(dfs[3], is_train)
    med_data  = cleanup_meds(dfs[4],  is_train)
    img_data  = cleanup_img(dfs[5],   is_train)
    proc_data = cleanup_procs(dfs[6], is_train)
    cnd_data  = cleanup_cnds(dfs[7],  is_train)
    imm_data  = cleanup_immns(dfs[8], is_train)   
    
    data_tables = [pt_data[0], pt_data[1], obs_data[0], alg_data[0], crpl_data[0], med_data[0], img_data[0], proc_data[0], cnd_data[0], imm_data[0]]
    if is_train:
        code_tables = [pt_data[2], obs_data[1], alg_data[1], crpl_data[1], med_data[1], img_data[1], proc_data[1], cnd_data[1], imm_data[1]]
    
    return (data_tables, code_tables) if is_train else (data_tables)

In [ ]:
data_tables, code_tables = cleanup_dataset(f'{PATH_1K}/raw_split/train', is_train=True)

patients, pt_demographics, observations, allergies, \
careplans, medications, imaging_studies, procedures, conditions, immunizations = data_tables

pt_codes, obs_codes, alg_codes, crpl_codes, med_codes, img_codes, proc_codes, cnd_codes, imm_codes = code_tables

In [ ]:
conditions.count()

date    7666
code    7666
dtype: int64

In [ ]:
obs_codes.count()

orig_code    173312
desc         173312
value        173312
units        173312
type         173312
dtype: int64

## Extract Labels (y)

The labels we intend to predict are conditions and must be in the `CONDITIONS` dict
- Adding them to the `patients` df
- And adding the patient's age when the particular condition was recorded

In [ ]:
for key in CONDITIONS.keys():
    print(key,"::",CONDITIONS[key])

diabetes :: 44054006||START
stroke :: 230690007||START
alzheimers :: 26929004||START
coronary_heart :: 53741008||START


In [ ]:
#export
def extract_ys(patients, conditions, cnd_dict=CONDITIONS):
    '''Extract labels from conditions df and add them to patients df with age'''
    for key in cnd_dict.keys():
        patients = patients.merge(conditions[conditions.code==cnd_dict[key]], how='left', left_index=True, right_index=True)
        patients[f'{key}_y'] = patients.code.notna()
        patients[f'{key}_age'] = ((patients.date - patients.birthdate)//np.timedelta64(1,'Y'))
        patients = patients.drop(columns=['date','code'])
    return patients

In [ ]:
tmp_pts = extract_ys(patients, conditions, CONDITIONS)

In [ ]:
tmp_pts.count()

birthdate             702
diabetes_y            702
diabetes_age           43
stroke_y              702
stroke_age             30
alzheimers_y          702
alzheimers_age         12
coronary_heart_y      702
coronary_heart_age     39
dtype: int64

## Insert Age

Inserting patient's age in months and years into each record df
- this can be modified to records the patient's age in **days** or even **hours** that might be more relevant for datasets involving hospitalizations or ER admissions 

In [ ]:
#export
def insert_age(df, pts_df):
    '''Insert age in years and months into each of the rec dfs'''
    df = df.merge(pts_df, left_index=True, right_index=True)
    df['age']        = (df['date'] - df['birthdate'])//np.timedelta64(1,'Y')
    df['age_months'] = (df['date'] - df['birthdate'])//np.timedelta64(1,'M')
    return df.drop(columns=['date','birthdate'])

## Do-All Functions
The actual functions that will be called from other modules

In [ ]:
#export
def clean_raw_ehrdata(path, valid_pct=0.2, test_pct=0.2, today=None):
    '''Split, clean, preprocess & save raw EHR data'''
    split_ehr_dataset(path, valid_pct, test_pct)
    
    for split in ['train', 'valid', 'test']:
        split_path = f'{path}/raw_split/{split}'
        if split == 'train': data_tables, code_tables = cleanup_dataset(split_path, is_train=True, today=today)
        else               : data_tables = cleanup_dataset(split_path, is_train=False)
        patients, conditions, rec_tables = data_tables[0], data_tables[8], data_tables[2:]
        patients = extract_ys(patients, conditions, CONDITIONS)
        rec_dfs = [insert_age(rec_df, pd.DataFrame(patients.birthdate)) for rec_df in rec_tables]
        
        cleaned_dir = Path(f'{path}/cleaned/{split}')
        cleaned_dir.mkdir(parents=True, exist_ok=True)
    
        for rec_df,name in zip(rec_dfs,FILENAMES[1:]):
            rec_df.to_csv(f'{cleaned_dir}/{name}.csv')
        patients.reset_index(inplace=True)
        patients.to_csv(f'{cleaned_dir}/patients.csv', index_label='indx')
        data_tables[1].to_csv(f'{cleaned_dir}/patient_demographics.csv')
        print(f'Saved cleaned "{split}" data to {cleaned_dir}')
        
        if split == 'train':
            codes_dir = Path(f'{cleaned_dir}/codes')
            codes_dir.mkdir(parents=True, exist_ok=True)
            for code_df,name in zip(code_tables, FILENAMES):
                code_df.to_csv(f'{codes_dir}/code_{name}.csv', index_label='indx')
            print(f'Saved vocab code tables to {codes_dir}')

In [ ]:
#export
def load_cleaned_ehrdata(path):
    '''Load cleaned, age-filtered EHR data'''
    
    csv_names = FILENAMES.copy()
    csv_names.insert(1,'patient_demographics')
    
    train_dfs = [pd.read_csv(f'{path}/cleaned/train/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
    valid_dfs = [pd.read_csv(f'{path}/cleaned/valid/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
    test_dfs  = [pd.read_csv(f'{path}/cleaned/test/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
                             
    return train_dfs, valid_dfs, test_dfs

In [ ]:
#export
def load_ehr_vocabcodes(path):
    '''Load codes for vocabs'''
    
    code_dfs = [pd.read_csv(f'{path}/cleaned/train/codes/code_{fname}.csv', low_memory=False, na_filter=False, index_col=0) for fname in FILENAMES]
                             
    return code_dfs

In [ ]:
clean_raw_ehrdata(PATH_1K)

Splits:: train: 0.6, valid: 0.2, test: 0.2
Split patients into:: Train: 702, Valid: 234, Test: 235 -- Total before split: 1171
Saved train data to /home/vinod/.lemonpie/datasets/synthea/1K/raw_split/train
Saved valid data to /home/vinod/.lemonpie/datasets/synthea/1K/raw_split/valid
Saved test data to /home/vinod/.lemonpie/datasets/synthea/1K/raw_split/test
Saved cleaned "train" data to /home/vinod/.lemonpie/datasets/synthea/1K/cleaned/train
Saved vocab code tables to /home/vinod/.lemonpie/datasets/synthea/1K/cleaned/train/codes
Saved cleaned "valid" data to /home/vinod/.lemonpie/datasets/synthea/1K/cleaned/valid
Saved cleaned "test" data to /home/vinod/.lemonpie/datasets/synthea/1K/cleaned/test


In [ ]:
train_dfs, valid_dfs, test_dfs = load_cleaned_ehrdata(PATH_1K)
code_dfs = load_ehr_vocabcodes(PATH_1K)

In [ ]:
# for df in train_dfs:
#     display(df.head())

In [ ]:
thispt = train_dfs[0].iloc[0]

In [ ]:
thispt

patient               b1d50391-79c5-403c-919f-3ded66c9d77a
birthdate                                       1959-09-01
diabetes_y                                           False
diabetes_age                                           NaN
stroke_y                                             False
stroke_age                                             NaN
alzheimers_y                                         False
alzheimers_age                                         NaN
coronary_heart_y                                     False
coronary_heart_age                                     NaN
Name: 0, dtype: object

In [ ]:
thispt['patient'], thispt['birthdate']

('b1d50391-79c5-403c-919f-3ded66c9d77a', '1959-09-01')

In [ ]:
thispt['diabetes_y'], thispt['stroke_y'], thispt['alzheimers_y'], thispt['coronary_heart_y']

(False, False, False, False)

In [ ]:
# for df in code_dfs:
#     display(df.head())

Making sure condition counts match - before and after split

In [ ]:
CONDITIONS

{'diabetes': '44054006||START',
 'stroke': '230690007||START',
 'alzheimers': '26929004||START',
 'coronary_heart': '53741008||START'}

In [ ]:
pts_train = train_dfs[0]
cnd_train, cnd_valid, cnd_test = train_dfs[8], valid_dfs[8], test_dfs[8]

In [ ]:
len(cnd_train[cnd_train['code'] == '44054006||START'])

43

In [ ]:
len(pts_train[pts_train['diabetes_y'] == 1])

43

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_basics.ipynb.
Converted 01_preprocessing_clean.ipynb.
Converted 02_preprocessing_vocab.ipynb.
Converted 03_preprocessing_transform.ipynb.
Converted 04_data.ipynb.
Converted 05_metrics.ipynb.
Converted 06_learn.ipynb.
Converted 07_models.ipynb.
Converted 08_experiment.ipynb.
Converted 99_quick_walkthru.ipynb.
Converted 99_running_exps.ipynb.
Converted index.ipynb.
